# imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import compress

from src.io import DataLoader
from src.preproc import Preprocessor

# prep data

In [3]:
dl = DataLoader()
dataset = dl.load_raw()

In [4]:
preprocessor = Preprocessor()
dataset_preproc = preprocessor(dataset)

In [5]:
hate = [t==1 for t in dataset_preproc['train']['target']]

In [6]:
list(compress(dataset_preproc['test']['sentence'], hate))[:10]

['Czy Adam już nie żyje? Jeśli tak, to jak rozwiązali jego wątek?  #nadobreinazłe',
 '@anonymized_account @anonymized_account @anonymized_account W takim razie sie zapedzilem, i tak nie daje banów, nawet botom',
 '@anonymized_account Nieprawda. Normalnie tak się nie \\"tnie\\".',
 'Coś się dzieje atak botów z każdej strony',
 'na ból gardła najlepsza wódeczka',
 'RT @anonymized_account Może być kliknęli i zrobili dobry uczynek? Każdy RT = oddany na Marcina Gortata głos.\\n\\n@anonymized_account #NBAvote',
 '@anonymized_account @anonymized_account no nic pozostaje nam walka o wicemistrzostwo :(',
 'RT @anonymized_account @anonymized_account Chciałbyś aby tak było miernoto :)',
 '@anonymized_account @anonymized_account e, ja już jedną w tym roku przeczytałam, wystarczy.',
 '@anonymized_account Kiedyś lm96 w galerii w Wawie XDD']

In [7]:
list(compress(dataset_preproc['test']['preproc'], hate))[:10]

['Czy Adam już nie żyje? Jeśli tak, to jak rozwiązali jego wątek? nadobreinazłe',
 'W takim razie sie zapedzilem, i tak nie daje banów, nawet botom',
 'Nieprawda. Normalnie tak się nie tnie.',
 'Coś się dzieje atak botów z każdej strony',
 'na ból gardła najlepsza wódeczka',
 'RT Może być kliknęli i zrobili dobry uczynek? Każdy RT oddany na Marcina Gortata głos.nn NBAvote',
 'no nic pozostaje nam walka o wicemistrzostwo',
 'RT Chciałbyś aby tak było miernoto',
 'e, ja już jedną w tym roku przeczytałam, wystarczy.',
 'Kiedyś lm96 w galerii w Wawie XDD']

# translate

In [8]:
import sentencepiece

In [73]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-pl-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-pl-en")

In [106]:
translated = model.generate(**tokenizer(dataset_preproc['train']['preproc'][:100], return_tensors="pt", padding=True))

In [107]:
translated_en = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# classify

In [111]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer_class = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model_class = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [138]:
input_clf = tokenizer_class(translated_en, return_tensors="pt", padding=True)

In [153]:
classified = model_class(**input_clf)

In [147]:
classified['logits'][1][1]

tensor(3.4106, grad_fn=<SelectBackward>)

In [151]:
def logit2prob(logit: float) -> float:
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return(prob)

In [152]:
logit2prob(3.4106)

0.9680341741323084